---
date: 2024-11-04
comments: true
draft: true
tags:
    - Agent
    - LLMs
---

# Understanding and Building ReAct Agents from Scratch

## Introduction

## What is a ReAct Agent?

A ReAct Agent is an autonomous agent architecture that uses reasoning and action planning capabilities of LLMs for general task solving. Given a task and the tools to solve it, a ReAct agent dynamically plans and executes a sequence of actions to achieve the goal.

## Why is it required?

### Limitations of Chain of Thought Prompting

CoT promting is not good enough for dynamic problem solving for the following reasons:

1. CoT follows a fixed sequence and cannot dynamically adapt based on the current state of the task.
2. CoT prompting typically does not have any access to external tools.
3. It does not perform well over long sequences of reasoning as hallucinations are more prevalent in CoT since the model is not groudned in truth.

### Limitations of Act-Only Agents

Explainability: Act-only agents are not explainable. They are black-boxes that take in an input and produce an output. There is no way to understand why a particular action was taken. This is can be limiting especially when the agent prompt needs to be "debugged" and improved.

### How ReAct Agents Solve These Limitations

1. Dynamic Planning: ReAct agents can dynamically plan and execute a sequence of actions to solve a task based on the previous results.
2. ReAct Agents integrate external tools such as APIs, databases, calculators, etc. to solve tasks. This makes sure that they have access to latest information and more reliable results to work with.
3. Explainability: sequences of ReAct agents contains a reasoning/thought step which can be used to understand why a particular action was taken.
4. Reliability: With each sequence contain a reasoning, action, and observation step the agent is more grounded and trustworthy, since the each sequence is more structured and constrained.

## Pros and Cons of ReAct Agents

### Pros

1. Enhanced Problem Solving Abilities: Since ReAct agents can use external tools and also dynamically adapt, they can create a wide range of sophisticated and lengthy workflows to solve a probelm reliably.
2. Versatility: ReAct Agents can handle a variety of tasks as long as they have necessary tools to access information that aid them.
3. Access to new Information: ReAct agents can access external tools and APIs to get the latest information to solve a task. Whereas CoT and other methods are limited to the information the model was trained on or is present in the prompt.

### Cons

1. Requires Large Models: For ReAct agents to work even decently, they require larger language models such as GPT-3 or GPT-4. This means that if you are using an open source model, then the computational overhead will be high, and if you are using a closed source, then the API costs will be higher.

2. High Latency: ReAct agents require large models and also takes multiple sequences to solve a task. This means that the latency of the agent will be high. This can be a problem for real-time applications.

3. Simple ReAct agents setups may not be enough: The ReAct agent prompt setup, workflow and the tools need to be carefully designed to make sure that the agent is able to solve the task. Especially when the task is highly domain specific or requires a lot of external information. A simple plug-and-play setup from the LangChain/LlamaIndex may not be enough.

## Understanding the ReAct Agent Framework

The ReAct Agent Framework Diagram above shows the different components and the sequence in which they are used:

* **User Query**: The user query is the task that the ReAct agent needs to solve.
* **Tools Description**: This contains a description of each of the tools that is available to the ReAct agent. This can be APIs, databases, calculators, etc.
* **LLM**: The large language model is used to generate the reasoning and action steps.
* **Traces**: This is a list of steps takes by the React Agent to solve the task. Each step contains the following:
  * Thought: The LLM's reasoning about the query why a particular action was taken.
  * Action: The action that the ReAct agent takes to solve the task.
  * Observation: The output of the tool that the ReAct agent used to solve the task.
Along with the Tools Desceiptions, the Traces are sent to the LLM to generate the next step in the sequence. This new step is added back to the Traces and the process is repeated until a final satisfactory answer is obtained.
* **Tools**: The tools are the external APIs, databases, calculators, etc. that the ReAct agent has access to solve the task.
* **Tool Call and Tool Response**: The Tool Call is a specific query that contains the tool name and its parameters. The Tool Response is the output of that tool, which becomes the observation in the trace.
* **Result**: Once the LLM decides that a satisfactory result that answers the user's query has been obtained, the result is returned to the user. The LLM decides that result has been obtained based on the traces provided. 

## Building a ReAct Agent from Scratch

### Question Space

#### Current Weather Queries

1. What is the weather in New York?
2. What's the temperature in London?
3. Is it raining in Paris?

#### Weather Forecast Queries

1. What will be the weather in New York tomorrow?
2. How hot will it be in Dubai next week?
3. Will it snow in Moscow this weekend?

#### Activity Planning Queries

1. Can I go for a picnic in London tomorrow at 12pm?
2. Should I plan a beach day in Miami next week?
3. Can I go for a run in brooklyn on sunday morning?

### Tools

1. Realtime Weather Tool: This tool provides the current weather information for a given location.
2. Weather Forecast Tool: This tool provides the 3 day weather forecast for a given location for a given time.
3. Activity Planning Tool: This tool has a knowledge base using which it can answer questions about whether a particular activity can be done given the weather conditions, and provides recommendations accordingly.
4. Alert and Warning Tool: This tool provides alerts and warnings for a given location. 


In [5]:
import requests
import os

from dotenv import load_dotenv

load_dotenv("../../../.env")

def get_current_weather(city, aqi: bool = False): 
    url = f"http://api.weatherapi.com/v1/current.json"
    params = {
        "key": os.getenv("WEATHER_API_KEY"),
        "q": city,
        "aqi": aqi
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error: ", response.status_code)
        print(response.json()['error'])

    return None


def get_forecast(city, days: int = 3, aqi: bool = False, alerts: bool = False):
    url = f"http://api.weatherapi.com/v1/forecast.json"
    params = {
        "key": os.getenv("WEATHER_API_KEY"),
        "q": city,
        "days": days,
        "aqi": aqi,
        "alerts": alerts
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error: ", response.status_code)
        print(response.json()['error'])

    return None


weather = get_current_weather("new york", aqi=True)
print(weather)

forecast = get_forecast("new york", days=3, aqi=True, alerts=True)
print(forecast)


{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'lon': -74.0064, 'tz_id': 'America/New_York', 'localtime_epoch': 1734856404, 'localtime': '2024-12-22 03:33'}, 'current': {'last_updated_epoch': 1734856200, 'last_updated': '2024-12-22 03:30', 'temp_c': -6.3, 'temp_f': 20.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 14.3, 'wind_kph': 23.0, 'wind_degree': 314, 'wind_dir': 'NW', 'pressure_mb': 1025.0, 'pressure_in': 30.27, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 68, 'cloud': 0, 'feelslike_c': -12.9, 'feelslike_f': 8.8, 'windchill_c': -12.9, 'windchill_f': 8.8, 'heatindex_c': -6.3, 'heatindex_f': 20.6, 'dewpoint_c': -11.3, 'dewpoint_f': 11.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 17.5, 'gust_kph': 28.2}}
{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'l

### Tools Implementation